# The Fire Net Model trained on AWS SageMaker

## Using Depthwise Separable Convolution

This is my own model using depthwise separable convolutions. There is a version which has been designed before using normal standard convolution.

Therefore, for my project we have:
* A bench mark model which is a pre trained Mobile net.
* A fire net model made up of standard convolution.
* A fire net model made up of depthwise convolution.

In [10]:
%pip install -U sagemaker

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [11]:
import tensorflow as tf
import numpy as np
import os
import numpy as np 
import matplotlib.pyplot as plt
import glob
import shutil
import tensorboard
import datetime
import sagemaker
from sagemaker.tensorflow import TensorFlow

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, BatchNormalization, DepthwiseConv2D, AveragePooling2D
from sklearn.metrics import classification_report, confusion_matrix,roc_curve,auc, roc_auc_score,precision_recall_curve
from sklearn.metrics import PrecisionRecallDisplay,RocCurveDisplay,ConfusionMatrixDisplay

In [ ]:
#initial download to instance. After downloading to S3, no need to run this again.
_URL = 'https://fire-net-datasets.s3.amazonaws.com/Training_Dataset.zip'

zip_file = tf.keras.utils.get_file(origin=_URL,extract=True)  
#This will ge the file and extract it to a directory and extract to /Training Dataset

In [ ]:
print(os.path.dirname(zip_file))
#This function returns the directory of the extracted folder without the extracted folder inclusive

In [ ]:
#No need to run this after uploading to S3
base_dir = os.path.join(os.path.dirname(zip_file), 'Training Dataset')
#A good way to add the directory of the extracted folder and also the extracted folder itself.
print(base_dir)

In [ ]:
classes = ['Fire', 'NoFire']
sets = ['train', 'val']

In [ ]:
for cl in classes:
  img_path = os.path.join(base_dir, cl)
  images = glob.glob(img_path + '/*')
  print("{}: {} Images".format(cl, len(images)))
  train, val = images[:round(len(images)*0.7)], images[round(len(images)*0.7):]

  for t in train:
    if not os.path.exists(os.path.join(base_dir, 'train', cl)):
      os.makedirs(os.path.join(base_dir, 'train', cl))
    shutil.move(t, os.path.join(base_dir, 'train', cl))

  for v in val:
    if not os.path.exists(os.path.join(base_dir, 'val', cl)):
      os.makedirs(os.path.join(base_dir, 'val', cl))
    shutil.move(v, os.path.join(base_dir, 'val', cl))


In [ ]:
sets_counts = {
    'train': 0,
    'val': 0
}

for set_name in sets:
    for class_name in classes:
        path = os.path.join(base_dir, set_name, class_name)
        count = len(os.listdir(path))
        print(path, 'has', count, 'images')
        sets_counts[set_name] += count

print(sets_counts)

In [13]:
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
dataset_bucket = "swifty-datasets"

bucket = sess.default_bucket()
prefix = "firenet_aws"
tensorflow_logs_path = "s3://{}/{}/logs".format(bucket, prefix)

print("Bucket: {}".format(bucket))
print("SageMaker ver: " + sagemaker.__version__)
print("Tensorflow ver: " + tf.__version__)

Bucket: sagemaker-us-east-1-038469568353
SageMaker ver: 2.84.0
Tensorflow ver: 2.6.2


In [ ]:
#Do not run this when data already in S3
print("Uploading to S3")
s3_data_path = sess.upload_data(path=base_dir, bucket=dataset_bucket, key_prefix='firenet_data')
print("Uploaded to", s3_data_path)

In [14]:
#Instead make it this:
s3_data_path = "s3://swifty-datasets/firenet_data"

In [15]:
keras_metric_definition = [
    {"Name": "train:loss", "Regex": ".*loss: ([0-9\\.]+) - accuracy: [0-9\\.]+.*"},
    {"Name": "train:accuracy", "Regex": ".*loss: [0-9\\.]+ - accuracy: ([0-9\\.]+).*"},
    {
        "Name": "validation:accuracy",
        "Regex": ".*step - loss: [0-9\\.]+ - accuracy: [0-9\\.]+ - val_loss: [0-9\\.]+ - val_accuracy: ([0-9\\.]+).*",
    },
    {
        "Name": "validation:loss",
        "Regex": ".*step - loss: [0-9\\.]+ - accuracy: [0-9\\.]+ - val_loss: ([0-9\\.]+) - val_accuracy: [0-9\\.]+.*",
    },
    {
        "Name": "sec/steps",
        "Regex": ".* (\d+)[mu]s/step - loss: [0-9\\.]+ - accuracy: [0-9\\.]+ - val_loss: [0-9\\.]+ - val_accuracy: [0-9\\.]+",
    },
]

In [18]:
hyper_parameters = {"epochs": 50, "tf-logs-path": tensorflow_logs_path}
#ml.g4dn.xlarge GPU at 0.96 per hour
estimator = TensorFlow(
    entry_point='train.py',
    base_job_name="firenet-training",
    source_dir="source_dir",
    role=role,
    instance_type='ml.g4dn.xlarge',
    instance_count=1,
    py_version='py38',
    framework_version = '2.6.2',
    hyperparameters =hyper_parameters,
    metric_definitions=keras_metric_definition,
    output_path='s3://swifty-ai-models/other_models/firenet_tf_sm'
    
)

In [ ]:
estimator.fit(s3_data_path, wait=False)

In [ ]:
fire_predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')
print('\nModel is deployed')

In [ ]:
#Upload images to the test directory first.
test_dir = 'Fire_Detection/data/test/'
test_images = [os.path.join(test_dir, x) for x in os.listdir(test_dir)]
print(test_images[0])


In [ ]:
def get_pred(img_path):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(128, 128))
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    results = fire_predictor.predict(img)
    return results

In [ ]:
#Getting predictions for all images
def showPredictions(image_list):
    predicted_classes = []
    for i in range(len(image_list)):
        image_path = image_list[i]
        results = get_pred(image_path)
        print(results)
        predicted_id = np.argmax(results)
        predicted_class = classes[predicted_id]
        predicted_classes.append(predicted_class)
    plt.figure(figsize=(10,9))
    for n in range(len(image_list)):
        plt.subplot(6,5,n+1)
        plt.subplots_adjust(hspace = 0.3)
        image = plt.imread(image_list[n])
        plt.imshow(image)
        plt.title(predicted_classes[n].title())
        plt.axis('off')

In [ ]:
sagemaker_session.delete_endpoint(fire_predictor.endpoint)

In [19]:
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)

hyperparameter_ranges = {
    "learning-rate": ContinuousParameter(0.00001, 0.001),
    "batch-size": CategoricalParameter([64, 128]),
    "optimizer": CategoricalParameter(["sgd", "adam", "rmsprop"]),
}

objective_metric_name = "validation:accuracy"


tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions=keras_metric_definition,
    objective_type="Maximize",
    max_jobs=6,
    max_parallel_jobs=2,
    early_stopping_type="Auto",
    base_tuning_job_name="firenet-hpo-tuning",
)

tuner.fit(s3_data_path)

ResourceLimitExceeded: An error occurred (ResourceLimitExceeded) when calling the CreateHyperParameterTuningJob operation: The account-level service limit 'ml.g4dn.xlarge for training job usage' is 0 Instances, with current utilization of 0 Instances and a request delta of 2 Instances. Please contact AWS support to request an increase for this limit.

In [ ]:
%pip install tensorboard

Now that we have got the best model, we can deploy it to an endpoint. Please refer to other SageMaker sample notebooks or SageMaker documentation to see how to deploy a model.

Paste the command that is the output of the next cell to start your tensorboard instance on Studio:

In [ ]:
aws_region = sess.boto_region_name
!AWS_REGION={aws_region}
!echo tensorboard --logdir {tensorflow_logs_path}

Instance of TensorBoard will be available at https://<notebook instance hostname>/proxy/6006/. By default TensorBoard assigns port 6006, but if it’s already in use TensorBoard will increase the port by 1, so 6007, 6008 and so on until it finds an available port.

# Link to tensorboard

https://d-aten2bwlosyw.studio.us-east-1.sagemaker.aws/jupyter/default/proxy/6006/